# Esercizio 3 - Sequence tree

utility


In [1]:
from datetime import timedelta
from datetime import datetime

def format_timedelta(td: timedelta) -> str:
    days = td.days
    years, days = divmod(days, 365)
    months, days = divmod(days, 30)
    hours, remainder = divmod(td.seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    
    formatted_str = ""
    if years:
        formatted_str += f"{years}y "
    if months:
        formatted_str += f"{months}mo "
    if days:
        formatted_str += f"{days}d "
    if hours:
        formatted_str += f"{hours}h "
    if minutes:
        formatted_str += f"{minutes}m "
    if seconds:
        formatted_str += f"{seconds}s "
    

    
    return formatted_str[:-1] if formatted_str else "0s"

def print_dataset_state(dataset_X,dataset_Y,dataset_ST,dataset_V,dataset_I,indexes=None):
    if (indexes and len(indexes)>len(dataset_X)):
        indexes=None
    print("file\tindex\tX\tY\tST\t\t\tV")
    if indexes is not None:
        for i in indexes:
            print(f"{dataset_I[i]}\t{i}{len(dataset_X[i])}\t{dataset_Y[i]}\t{dataset_ST[i]}\t{dataset_V[i]}")
        print(f"# entries: {len(indexes)}")

    else:
        for i in range(0,len(dataset_X)):
            print(f"{dataset_I[i]}\t{i}\t{len(dataset_X[i])}\t{dataset_Y[i]}\t{dataset_ST[i]}\t{dataset_V[i]}")
        print(f"# entries: {len(dataset_X)}")
    
def find_durations(dataset):
    lengths = []
    for entry in dataset:
        min_t = datetime.max
        max_t = datetime.min
        for item in entry:
            if item[0] < min_t:
                min_t = item[0]
            elif item[0] > max_t:
                max_t = item[0]
        lengths.append(max_t-min_t)
    return [(i, x) for i, x in enumerate(lengths)]

## Operazioni preliminari sul DS

#### Carichiamo il DS

In [3]:
import os
from datetime import datetime

def load_diabetes_dataset(verbose=False)-> list: 
    folder_path="datasets\\diabetes"
    dataset = []
    errcount=0

    for filename in os.listdir(folder_path):

        file_path = os.path.join(folder_path, filename)

        if os.path.isfile(file_path)and filename.startswith('data'):
            entry=[]

            with open(file_path, 'r') as file:
                content = file.readlines()
                for line in content:
                    item = tuple((line[0:-1] if line.endswith('\n') else tuple(line)).split("\t"))

                    # If the item is valid, append it to the entry
                    try:
                        item_f = datetime.strptime(item[0]+" "+item[1], "%m-%d-%Y %H:%M")
                        entry.append((item_f,item[2],item[3]))
                    except:
                        if(verbose):
                            print(f"[!] Entry {item} in file {filename} is NOT vallid. Skipped!")
                        errcount+=1
                # add the entry to the dataset
                dataset.append(entry)
    print(f"Skipped {errcount} items for formatting issues in data file. {len(dataset)} loaded.")
    return dataset
dataset = load_diabetes_dataset(False)
#[(i,format_timedelta(x)) for (i,x) in find_durations(dataset)]


Skipped 46 items for formatting issues in data file. 70 loaded.


#### A partire dal dataset grezzo generiamo X,Y,ST e V.

> Definiamo la classe reale come uno [0,1] che indica se l'evento 65 (65 = Hypoglycemic symptoms) si è verificato in una certa finestra di tempo `event_window` dopo un certo waiting time `waiting_window`.

In [4]:
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta

def compute_datasets(dataset:list,observation_window,waiting_window,prediction_window)->list:

    dataset_X = []
    dataset_Y = []
    dataset_F = []

    dataset_ST = [entry[0][0] for entry in dataset ]

    count_excluded=0

    for i in range(0,len(dataset)):
        entry = dataset[i]

        end_obs = dataset_ST[i]+observation_window
        
        start_pred = end_obs + waiting_window
        end_pred = start_pred + prediction_window

        if end_pred <= entry[-1][0]:
            entry_X = []
            found = 0

            for item in entry:
                if item[0]>= dataset_ST[i] and item[0]<end_obs:
                    entry_X.append(item)
                if item[0]>=start_pred and item[0]<end_pred:
                    # put Y=1 if it has at least one "65" entry
                    if (item[1]=="65"):
                        found = 1
            dataset_X.append(entry_X)
            dataset_Y.append(found)
            dataset_F.append(i+1)
        else:
            count_excluded+=1
    
    dataset_ST = [entry[0][0] for entry in dataset_X ]
    dataset_V = [None]*len(dataset_X)

    print(f"{count_excluded} entries unsuitable for selected windows.")
    print(f"Final dataset lengths: {len(dataset_X),len(dataset_Y),len(dataset_ST),len(dataset_V),len(dataset_F)}")    
    return dataset_X,dataset_Y,dataset_ST,dataset_V,dataset_F

dataset = load_diabetes_dataset()
observation_window = timedelta(days=+5)
waiting_window = timedelta(days=+5)
prediction_window = timedelta(days=+15)


dataset_X,dataset_Y, dataset_ST,dataset_V, dataset_F = compute_datasets(dataset[:15],observation_window,waiting_window,prediction_window)



Skipped 46 items for formatting issues in data file. 70 loaded.
1 entries unsuitable for selected windows.
Final dataset lengths: (14, 14, 14, 14, 14)


#### Stampiamo il DS elaborato.

In [5]:
def print_dataset_state(dataset_X,dataset_Y,dataset_ST,dataset_V,dataset_F,indexes=None):
    if (indexes and len(indexes)>len(dataset_X)):
        indexes=None
    
    print("file\tindex\tX\tY\tST\t\t\tV")

    if indexes is None:
        indexes = range(0,len(dataset_X))

    for i in indexes:
        print(f"{dataset_F[i]}\t{i}\tl.{len(dataset_X[i])}\t{dataset_Y[i]}\t{dataset_ST[i]}\t{dataset_V[i]}")
    print(f"# entries: {len(dataset_X)}")

print_dataset_state(dataset_X,dataset_Y, dataset_ST,dataset_V,dataset_F)

file	index	X	Y	ST			V
1	0	l.35	1	1991-04-21 09:09:00	None
2	1	l.38	0	1989-10-10 08:00:00	None
3	2	l.49	1	1990-07-21 06:43:00	None
4	3	l.41	0	1990-08-19 17:00:00	None
5	4	l.46	0	1990-09-01 16:48:00	None
7	5	l.38	0	1989-03-27 22:00:00	None
8	6	l.24	1	1990-07-31 12:09:00	None
9	7	l.43	0	1990-04-22 18:08:00	None
10	8	l.34	0	1989-02-18 08:00:00	None
11	9	l.34	1	1990-07-13 09:44:00	None
12	10	l.46	1	1990-07-22 09:53:00	None
13	11	l.58	1	1990-09-04 05:53:00	None
14	12	l.38	1	1991-03-11 18:15:00	None
15	13	l.27	1	1991-04-13 08:47:00	None
# entries: 14


In [70]:
import itertools
import random
import math

random.seed(1)

# Find al possible d,l couples that I could split the tree on. Note: Ds are randomly selected bc otherwise I'd end up with ~36000 pairs...
def create_pairs(dataset_X:list,dataset_ST:list,howmany_d =30,random_sampling=False):

    #1. Create all labels available from current
    labels = set()
    for i in range(0,len(dataset_X)):
        for item in dataset_X[i]:
            if (item[0] > dataset_ST[i]):# Only consider label if it's not been superato
                labels.add(item[1])


    #2. Find all d
    durations = set()
    if random_sampling:
        durations = set()
        for i in range(0,len(dataset_X)):
            for item in dataset_X[i]:
                if (item[0] > dataset_ST[i]): # Only consider timestamp if it's not been superato
                    durations.add(item[0]-dataset_ST[i])
        print(len("aaaaaaaaaaaa "),durations)
        if len(durations)>howmany_d:
            durations = random.sample(sorted(durations),howmany_d) # Is this ok?
        else:
            durations = sorted(durations)
    else:
        durations = set()
        for i in range(0,len(dataset_X)):
            for entry in dataset_X[i]:
                durations.add(entry[0]-dataset_ST[i])

        delta = max(durations)/(howmany_d+1)
        durations = set()
        for i in range(1,howmany_d+1):
            durations.add(delta*i)

    #print([format_timedelta(x) for x in sorted(durations)])
    return sorted(list(itertools.product(durations,labels)))

#[print(format_timedelta(dl[0]),"->",dl[1],"|",end="") for dl in create_pairs(dataset_X,dataset_ST,30,False)]
#print("")
#[print(format_timedelta(dl[0]),"->",dl[1],"|",end="") for dl in create_pairs(dataset_X,dataset_ST,30,True)]
#print("")
#print(len(create_pairs(dataset_X,dataset_ST,30,False)),len(create_pairs(dataset_X,dataset_ST,30,True)))

# Given a set with binary classes, computes entropy
def compute_entropy(dataset_Y):
    ones = len(list(filter(lambda classification : classification == 1,dataset_Y)))
    zeros = len(list(filter(lambda classification : classification == 0,dataset_Y)))

    if(ones == 0 or zeros==0):
        return 0
    
    entropy = ones/len(dataset_Y)*math.log2(1/(ones/len(dataset_Y))) + zeros/len(dataset_Y)*math.log2(1/(zeros/len(dataset_Y)))

    return entropy

def compute_entropy_2(dataset_Y,dataset_Yt,dataset_Yf):
    return (len(dataset_Yt)/len(dataset_Y))*compute_entropy(dataset_Yt) + (len(dataset_Yf)/len(dataset_Y))*compute_entropy(dataset_Yf)

def test_event(dataset_X,dl_pair):
    i_T = [] # indexes of entries that have label==l within d time
    i_F = [] # indexes of entries that have DON'T HAVE label==l within d time
    d,l = dl_pair


    #1. Separate entries that satisfy event test from those who don't
    for i in range(0,len(dataset_X)):
        entry = dataset_X[i]

        found=False

        for item in entry:
            #starting from the starting time, see if it exists an item with timestamp < d and label == l
            if(not found and item[0]>= dataset_ST[i]): # If i'm over starting time
                if(item[0]< dataset_ST[i]+d and item[2]==l):
                    found=True
        if(found):
            i_T.append(i)
        else:
            i_F.append(i)





    return i_T,i_F

# Given a pair of duration d and label l, computes its information gain on the dataset if we were to split it according to the sequence tree rules.
def compute_IG(dl_pair,dataset_X,dataset_Y,verbose=False):
    #if verbose:
     #   print(f"Computing IG for {dl_pair}")

    entropy_0 = compute_entropy(dataset_Y)

    i_T, i_F = test_event(dataset_X,dl_pair)
    
    #2. Compute final entropy. first let's generate our new datasets...
    dataset_Yt=[ dataset_Y[i] for i in i_T]
    dataset_Yf=[ dataset_Y[i] for i in i_F]
            
    entropy_f = (len(i_T)/len(dataset_X))*compute_entropy(dataset_Yt) + (len(i_F)/len(dataset_X))*compute_entropy(dataset_Yf)

    if verbose:
        print(f"information gain is {dataset_Y} {entropy_0}-> {[ dataset_Y[i] for i in i_T],[ dataset_Y[i] for i in i_F]} {entropy_f}")
        
    return(entropy_0-entropy_f,i_T,i_F)




# Given all possible pairs of d,l finds the one with the highest information gain (aka the one I should actually split on)
def maximize_IG(dataset_X,dataset_Y, dataset_ST, indexes,verbose=False,howmany=30,random_sampling=False):
    dl_pairs = create_pairs([dataset_X[i] for i in indexes],[dataset_ST[i] for i in indexes],howmany,random_sampling)

    igs_dict = {x:compute_IG(x,dataset_X,dataset_Y,verbose) for x in dl_pairs[:]}

    max_dl = (max(igs_dict, key=igs_dict.get),igs_dict[max(igs_dict, key=igs_dict.get)])

    if len(max_dl[1][1])==0 or len(max_dl[1][2])==0:
        if verbose:
            print("Split failed, couldn't find a d,l that separates values :(")
        return None, 0
    
    if verbose:
        print("Max IG is in couple d=",format_timedelta(max_dl[0][0]),", l=",max_dl[0][1],", IG=",max_dl[1])

    return max_dl




print_dataset_state(dataset_X,dataset_Y, dataset_ST,dataset_V,dataset_F)
max_dl, max_ig=maximize_IG(dataset_X,dataset_Y,dataset_ST,range(0,len(dataset_X)),True,30,False)




file	index	X	Y	ST			V
1	0	l.35	1	1991-04-21 09:09:00	None
2	1	l.38	0	1989-10-10 08:00:00	None
3	2	l.49	1	1990-07-21 06:43:00	None
4	3	l.41	0	1990-08-19 17:00:00	None
5	4	l.46	0	1990-09-01 16:48:00	None
7	5	l.38	0	1989-03-27 22:00:00	None
8	6	l.24	1	1990-08-02 01:08:00	57
10	7	l.34	0	1989-02-18 08:00:00	None
12	8	l.46	1	1990-07-22 09:53:00	None
13	9	l.58	1	1990-09-04 05:53:00	None
14	10	l.38	1	1991-03-11 18:15:00	None
15	11	l.27	1	1991-04-13 08:47:00	None
16	12	l.22	1	1991-05-22 07:24:00	None
17	13	l.40	1	1990-07-13 09:48:00	None
18	14	l.44	1	1990-08-18 07:16:00	None
19	15	l.45	1	1990-09-09 17:23:00	None
20	16	l.39	0	1991-05-12 06:55:00	None
21	17	l.44	0	1989-09-03 08:00:00	None
22	18	l.36	0	1991-03-14 22:05:00	None
23	19	l.37	0	1991-04-27 23:02:00	None
24	20	l.28	0	1991-05-28 21:35:00	None
26	21	l.25	0	1988-07-13 08:00:00	None
27	22	l.16	0	1989-01-29 08:00:00	None
28	23	l.16	0	1989-11-05 07:00:00	None
29	24	l.40	0	1990-04-29 07:00:00	None
30	25	l.40	0	1990-12-18 07:00:00	None
31	26	l.3

## tree

In [145]:
from treelib import Node, Tree
class SequenceTree(Tree):
    def __init__(self, tree=None, deep=False, node_class=None, identifier=None):
        super(SequenceTree, self).__init__(tree=tree, deep=deep, node_class=node_class, identifier=identifier)

    def create_node(self, tag=None, identifier=None, parent=None, data=None,branch=None):
        """
        Create a child node for the given @parent node. If ``identifier`` is absent,
        a UUID will be generated automatically.
        """
        new_node = super(SequenceTree, self).create_node(tag=tag, identifier=identifier, parent=parent, data=data)
        
        siblings = super(SequenceTree,self).siblings(new_node.identifier)
        

        if len(super(SequenceTree,self).siblings(new_node.identifier))>=2:
           raise ValueError("Parent node already has maximum number of children")
        if branch in [x.data[0] for x in siblings]:
           raise ValueError(f"Parent node already has a {branch} branch")
        return new_node
    
    def display(self):
        print(self.show(stdout=False))
    
    def create_node_event(self,data,parent=None,branch=None):
        branch_f = "" if (branch is None) else branch+" "
        tag =  "\x1b[32m⬤\x1b[0m" +" "+ branch_f +"[" + str(data[1])+", " + format_timedelta(data[0])+ "] "  

        return     self.create_node(tag,data=(branch,{"classification":data}),parent=parent,branch=branch)


def create_node_event(tree,data,parent=None,branch=None,entropy="",size=0,index=""):
    branch_f = "" if (branch is None) else str(branch)+" "
    tag =  f"\x1b[32m⬤ {branch_f} ({str(data[1])},{format_timedelta(data[0])})\x1b[0m - e{entropy:4.2} {size} - {index}"

    return     tree.create_node(tag,data=(branch,{"classification":data}),parent=parent,branch=branch)

def create_node_class(tree,classification,parent=None,branch=None,entropy="",size=0,index=""):
    branch_f = "" if (branch is None) else str(branch)+" "
    tag =  f"\x1b[33m◆ {branch_f} {classification} \x1b[0m- e{float(entropy):2.2} {size} - {index}"

    return     tree.create_node(tag,data=(branch,{"classification":classification}),parent=parent,branch=branch)

def create_node_value(tree,data,parent=None,branch=None):
    branch_f = "" if (branch is None) else branch+" "
    tag =  "\x1b[31m■\x1b[0m" +" "+ branch_f +"[" + str(data[1])+", " + format_timedelta(data[0])+ "] "  

    print(data)
    return     tree.create_node(tag,data=(branch,{"value":data}),parent=parent,branch=branch)


#print(tree.show(stdout=False))

## let's start

In [132]:
# prepare
dataset = load_diabetes_dataset(False)
observation_window = timedelta(days=+50)
waiting_window = timedelta(days=+5)
prediction_window = timedelta(days=+5)
dataset_X,dataset_Y, dataset_ST,dataset_V, dataset_F = compute_datasets(dataset[:],observation_window,waiting_window,prediction_window)

print(f"\nStarting entropy: {compute_entropy(dataset_Y):4.3}; classes: {sum(1 for c in dataset_Y if c == 1)}|{sum(1 for c in dataset_Y if c == 0)}")
print_dataset_state(dataset_X,dataset_Y,dataset_ST,dataset_V,dataset_F)


Skipped 46 items for formatting issues in data file. 70 loaded.
50 entries unsuitable for selected windows.
Final dataset lengths: (20, 20, 20, 20, 20)

Starting entropy: 0.469; classes: 2|18
file	index	X	Y	ST			V
1	0	l.348	1	1991-04-21 09:09:00	None
2	1	l.397	0	1989-10-10 08:00:00	None
20	2	l.366	0	1991-05-12 06:55:00	None
21	3	l.410	0	1989-09-03 08:00:00	None
26	4	l.138	0	1988-07-13 08:00:00	None
27	5	l.56	0	1989-01-29 08:00:00	None
28	6	l.223	0	1989-11-05 07:00:00	None
29	7	l.392	0	1990-04-29 07:00:00	None
30	8	l.388	0	1990-12-18 07:00:00	None
31	9	l.381	0	1991-05-20 08:00:00	None
41	10	l.392	0	1990-12-16 08:00:00	None
42	11	l.417	0	1991-06-30 08:00:00	None
50	12	l.258	0	1990-08-03 06:31:00	None
54	13	l.380	0	1990-07-29 07:00:00	None
55	14	l.421	0	1991-03-01 08:00:00	None
56	15	l.435	0	1989-02-03 08:00:00	None
65	16	l.417	0	1989-04-17 06:35:00	None
67	17	l.409	1	1991-01-01 09:10:00	None
68	18	l.103	0	1988-03-27 08:00:00	None
70	19	l.296	0	1989-03-13 08:00:00	None
# entries: 20


In [148]:
tree = SequenceTree()

def perform_event_test(max_dl,indexes,dataset_X,dataset_ST,dataset_V,verbose=False):
    # divide dataset in t and f...
    i_T = []
    i_F = []

    d,l = max_dl
    old_dataset_ST=dataset_ST
    for i in range(0,len(dataset_X)):
        entry = dataset_X[i]

        found=False

        for item in entry:
            #starting from the starting time, see if it exists an item with timestamp < d and label == l
            if(not found and item[0]>= old_dataset_ST[i]): # If i'm over starting time
                if(item[0]< old_dataset_ST[i]+d and item[2]==l):
                    found=True
                    dataset_ST[i] = item[0]
                    dataset_V[i] = item[2]

        if(found):
            i_T.append(i)
        else:
            i_F.append(i)
    return i_T,i_F

def fit(dataset_X,dataset_Y,dataset_V,dataset_ST,parent=None,branch=None,depth=0,indexes=""):
    #1. get best event.
    max_dl, max_ig = maximize_IG(dataset_X,dataset_Y,dataset_ST,range(len(dataset_X)),False,howmany=30,random_sampling=False)


    # If all the classes are the same, make a class node
    if all(element == dataset_Y[0] for element in dataset_Y):
        print(f"{depth,branch} All classes are the same!")
        create_node_class(tree=tree,classification=dataset_Y[0],parent=parent,branch=branch,entropy=compute_entropy(dataset_Y),size=len(dataset_Y),index=indexes)
        return
    
    # If classes are different but no split value is found, make a class node with majority
    if max_dl is None:
        print(f"{depth,branch} Couldn't find split node!")

        # Split not possible, make a class node!
        y_1 = sum(1 for c in dataset_Y if c == 1)
        y_0 = sum(1 for c in dataset_Y if c == 0)
        estimated_class = 1 if y_1>y_0 else 0
        
        create_node_class(tree,estimated_class,parent,branch,compute_entropy(dataset_Y),len(dataset_Y),indexes)
    
    else:
        node = create_node_event(tree,max_dl,parent,branch,compute_entropy(dataset_Y),len(dataset_Y),indexes)
        i_T,i_F=perform_event_test(max_dl,indexes,dataset_X,dataset_ST,dataset_V)
        #print(i_T,i_F)
        fit([dataset_X[i] for i in i_T],[dataset_Y[i] for i in i_T],[dataset_V[i] for i in i_T],[dataset_ST[i] for i in i_T],node,"t",depth+1,i_T)
        fit([dataset_X[i] for i in i_F],[dataset_Y[i] for i in i_F],[dataset_V[i] for i in i_F],[dataset_ST[i] for i in i_F],node,"f",depth+1,i_F)
        


fit(dataset_X,dataset_Y,dataset_V,dataset_ST,indexes=range(0,len(dataset_Y)))

if len(tree.all_nodes()) != 0:
    print(tree)

(1, 't') All classes are the same!
(1, 'f') Couldn't find split node!
⬤  (0,1d 14h 41m 29s) - e0.47 20 - range(0, 20)
├── ◆ f  0 - e0.5 18 - [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19]
└── ◆ t  0 - e0.0 2 - [2, 12]



In [76]:
# prepare
dataset = load_diabetes_dataset()
observation_window = timedelta(days=+5)
waiting_window = timedelta(days=+5)
prediction_window = timedelta(days=+15)
dataset_X,dataset_Y, dataset_ST,dataset_V, dataset_F = compute_datasets(dataset[:2],observation_window,waiting_window,prediction_window)

max_dl = maximize_IG(dataset_X,dataset_Y,dataset_ST,range(0,len(dataset_X)),False,30,False)


print(max_dl)


#rootnode = sequencetree.create_node_event(max_dl,None,None)

i_T,i_F = perform_event_test(max_dl,range(len(dataset_X)),dataset_X,dataset_ST,dataset_V)



print_dataset_state(dataset_X,dataset_Y,dataset_ST,dataset_V,dataset_F,i_T)
print_dataset_state(dataset_X,dataset_Y,dataset_ST,dataset_V,dataset_F,i_F)





Skipped 46 items for formatting issues in data file. 70 loaded.
0 entries unsuitable for selected windows.
Final dataset lengths: (2, 2, 2, 2, 2)
(None, 0)


TypeError: unsupported operand type(s) for +: 'datetime.datetime' and 'NoneType'